In [1]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from ftplib import FTP_TLS
import tkinter as tk
from tkinter import messagebox

delta=dt.timedelta(days=1)

In [2]:
# Función para asignar los días de la semana a cada fecha, si es festivo se trata como un domingo
import holidays
co_holidays = holidays.Colombia()

def typedays(row,tipo):

     if tipo=='WeekDay':
          return row['Fecha'].weekday()
     
     elif tipo=='WeekMonth':
          return (row['Fecha'].day - 1) // 7 + 1
     
     elif tipo=='DayType':
          if row['Fecha'] in co_holidays:
               return 3
          elif row['Fecha'].weekday()==5:
               return 2
          elif row['Fecha'].weekday()==6:
               return 3
          else:
               return 1

In [3]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)
# folderpath=s_parentpath.joinpath(r'DatosXM\Despacho\\')

# Ruta General
sRutaDesp=r'C:\Información XM\\Publico\\'

sRutaPrint=s_mainpath.joinpath('Print')

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython\Demanda
c:\Alejo\Eje de Planeación


In [4]:
# Rango de análisis

# s_FechaIni='2022-11-19'
# s_FechaFin='2024-11-18'

s_FechaIni='2024-12-21'
s_FechaFin='2025-01-17'

tipod='day_w'

#Camniar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
d_FechaFin=dt.datetime.strptime(s_FechaFin,'%Y-%m-%d')

# Create a DataFrame of dates
df_dates = pd.DataFrame(pd.date_range(start=s_FechaIni, end=s_FechaFin, freq='D')  , columns=["Date"])

df_dates['Mes']=df_dates['Date'].dt.month

df_dates

,Date,Mes
0,2024-12-21,12
1,2024-12-22,12
2,2024-12-23,12
3,2024-12-24,12
4,2024-12-25,12
5,2024-12-26,12
6,2024-12-27,12
7,2024-12-28,12
8,2024-12-29,12
9,2024-12-30,12


In [5]:
# Función para descargar el archivo
def DownFile(fecha_dt,UsuXM,PwsXM,Tipo):

    # Connect to the FTP server (replace with your actual details)
    ftps  = FTP_TLS()
    ftps .connect('xmftps.xm.com.co', 210)  # default port is 210

    # Secure the control connection
    ftps .auth()
    ftps .prot_p()  # Switch to secure data connection (important!)

    ftps .login(UsuXM, PwsXM)

    # Obtener mes y día de la fecha inicial
    # Transformar string en fecha
    # fecha = dt.datetime.strptime(fecha_dt, "%d/%m/%Y")

    # Obtener mes y día
    year= fecha_dt.year
    mes = fecha_dt.month
    dia = fecha_dt.day

    if Tipo=='DemDespacho':
        # Navigate to the directory you want to access
        ftps.cwd(rf"/INFORMACION_XM/Publico/DESPACHO/{year:04d}-{mes:02d}")
    elif Tipo=='Demanda':
        # Calcular el lunes asociado a la semana de la fecha dada
        lunes_asociado = fecha_dt - dt.timedelta(days=fecha_dt.weekday())
        # Obtener mes y día
        year= lunes_asociado.year
        mes = lunes_asociado.month
        dia = lunes_asociado.day
        ftps.cwd(rf"/INFORMACION_XM/PUBLICO/DEMANDAS/Pronostico Oficial/{year:04d}-{mes:02d}")
    else:
        messagebox.showinfo('Estado del proceso',f'No se reconoce el formato {Tipo}', parent=root)
        df=pd.DataFrame()
        return df


    # List files
    files = ftps.nlst()
    # print("Available files:", files)


    if Tipo=='DemDespacho':
        # Download condiciones iniciales de planta
        pathfile=rf"C:\Información XM\Publico\DESPACHO\{year:04d}-{mes:02d}"
        if not os.path.exists(pathfile):
            os.makedirs(pathfile)
        filename = rf"dDEM{mes:02d}{dia:02d}.TXT"    
    elif Tipo=='Demanda':
        # Calcular el lunes asociado a la semana de la fecha dada
        lunes_asociado = fecha_dt - dt.timedelta(days=fecha_dt.weekday())
        # Obtener mes y día
        year= lunes_asociado.year
        mes = lunes_asociado.month
        dia = lunes_asociado.day

        pathfile=rf"C:\Información XM\PUBLICO\DEMANDAS\Pronostico Oficial\{year:04d}-{mes:02d}"
        if not os.path.exists(pathfile):
            os.makedirs(pathfile)
        filename = rf"PRON_AREAS{mes:02d}{dia:02d}.TXT"

    try:
        # print(pathfile + "\\" + filename)
        with open(pathfile + "\\" + filename, 'wb') as f:
            ftps.retrbinary(f"RETR {filename}", f.write)

        ftps.quit()
        # print(f"{filename} downloaded successfully.")


    except:

        df=pd.DataFrame()

    
    return 

In [6]:
folderpath=Path(sRutaDesp + 'Despacho\\')

filetype='dDEM'

l_colDem=['Subarea']

l_SubCar=['ATLANTIC','BOLIVAR','GCM','CORDOSUC','CERROMAT'] # Lista con las subareas Caribe

# df_Factor=pd.DataFrame()
# df_Factor_typo_d=pd.DataFrame()

for i in range(1,25): 
    l_colDem.append(i)

df_Dem=pd.DataFrame()
df_datesAx=df_dates.copy()
df_datesAx=df_datesAx.sort_values(by='Date', ascending=True)

# print(mes)

for ind in df_datesAx.index:

    dFecha=df_datesAx.at[ind,'Date']

    ano=dFecha.year
    mes=dFecha.month
    dia=dFecha.day

    UsuXM='1060588666'
    PwsXM='Alejo230709*+'
    Tipo='DemDespacho'
    DownFile(dFecha,UsuXM,PwsXM,Tipo)

    
    file = filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
    s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
    df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colDem,encoding="ISO-8859-1")
    df_Aux['Fecha']=dFecha
    df_Aux = df_Aux[~df_Aux['Subarea'].isin(['Total','ECUADOR138','ECUADOR220','COROZO','CUATRIC'])]


    df_Dem=pd.concat([df_Dem,df_Aux])

df_DemAux=df_Dem.copy()

In [8]:
df_DemAux.to_csv(sRutaPrint.joinpath('D1.csv'))

In [18]:
df_Dem=df_DemAux.copy()

# Normalizar y reemplazar subáreas que no están en l_SubCar por "INTERIOR"
df_Dem['Subarea'] = df_Dem['Subarea'].astype(str).str.strip()
caribe_up = [s.upper() for s in l_SubCar]
mask = df_Dem['Subarea'].str.upper().isin(caribe_up)
df_Dem.loc[~mask, 'Subarea'] = 'INTERIOR'

# Definir tipo de día
df_Dem['day_osf']=df_Dem.apply(lambda row: typedays(row,tipo='DayType'),axis=1)
# Definir día de la semana
df_Dem['day_w']=df_Dem.apply(lambda row: typedays(row,tipo='WeekDay'),axis=1)
# Definir semana del mes
# df_Dem['week_m']=df_Dem.apply(lambda row: typedays(row,tipo='WeekMonth'),axis=1)

# Definir los periodos en una columna
df_Dem = pd.melt(df_Dem, id_vars=['Fecha','day_osf','day_w','Subarea'], var_name='Periodo', value_name='DemMW')


df_Dem['Mes']=df_Dem['Fecha'].dt.month
df_Dem['Dia']=df_Dem['Fecha'].dt.day

# Cálculo de la demanda por subárea
df_Dem=df_Dem.groupby(['Fecha','Mes','day_osf','day_w','Subarea','Periodo'])[['DemMW']].sum()
df_Dem=df_Dem.reset_index()
df_Dem['DemMW']=df_Dem['DemMW'].round(2)

# Ajustar nombres de Subarea según indicaciones
_mapping = {'ATLANTIC': 'SubAtlantico', 'BOLIVAR': 'SubBolivar', 'CERROMAT': 'SubCerromatoso', 'CORDOSUC': 'SubCordoba-Sucre','GCM': 'SubGCM','INTERIOR': 'SubAntioquia', }
df_Dem['Subarea'] = df_Dem['Subarea'].astype(str).apply(lambda s: _mapping.get(s.upper(), s))

df_Dem=df_Dem[['Fecha','Subarea','Periodo','DemMW']]

df_Dem['Fecha'] = df_Dem['Fecha'] + dt.timedelta(days=364)

df_Dem.to_csv(sRutaPrint.joinpath('DemandaDíaSub.csv'))


In [19]:
df_Dem

,Fecha,Subarea,Periodo,DemMW
0,2025-12-20,SubAtlantico,1,734.24
1,2025-12-20,SubAtlantico,2,713.01
2,2025-12-20,SubAtlantico,3,689.94
3,2025-12-20,SubAtlantico,4,670.26
4,2025-12-20,SubAtlantico,5,657.42
...,...,...,...,...
4027,2026-01-16,SubAntioquia,20,8051.87
4028,2026-01-16,SubAntioquia,21,7849.11
4029,2026-01-16,SubAntioquia,22,7129.73
4030,2026-01-16,SubAntioquia,23,6581.07
